In [20]:
import pickle
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
from qmc import QMC

In [18]:
d = pd.read_csv('./data/catsup_trainformat.csv', delimiter=',')
d.head(10)

,chosen,id,alt,display,feature,price,chid
0,0,1,0,0,0,4.6,0
1,0,1,1,0,0,3.7,0
2,1,1,2,0,0,5.2,0
3,0,1,3,0,0,3.4,0
4,0,1,0,0,0,4.6,1
5,0,1,1,0,0,4.3,1
6,1,1,2,0,0,5.2,1
7,0,1,3,0,0,4.4,1
8,0,1,0,0,0,4.6,2
9,0,1,1,0,1,2.5,2


In [19]:
X = d.values[:,1:-1]
X.shape

(11192, 5)

In [11]:
def summarize_data(model): 
    result = []
    for file in os.listdir('.\\resultaten'):
        if file.endswith('%s_results.p'%(model)):
            infile = open(os.path.join('.\\resultaten', file), 'rb')
            opt_list = pickle.load(infile)
            [result.append(i) for i in opt_list]
    
    pickle.dump( result, open( "./resultaten/%s_all_results.p" %(model), "wb" ) )
    return result      
        

In [ ]:
def probs(X,method,theta):
    #performs matrix product to obtain the probability of every row
    #X should be in format [display, feature, price]
    method = method[:2]
    R = method[4:6]
    if dist == 'QMC':
        delta = QMC(300,3,R)
    elif method == 'SMC':
        delta = np.random.randn(300,3,R)
    
    beta = theta[:K][:, None] + delta * theta[K:][:, None] ** 2
    print(beta.shape)

    try:
        assert(X.shape == (11192, 5) and beta.shape == (3,300))
        
    except AssertionError:
        raise AssertionError('Ga X ff in juiste format gooien. X: %s, beta: %s' %(X.shape,beta.shape))
    beta_choice = np.zeros((3,11192))
    for i in range(11192):
        id = int(X[i,0])
        beta_choice[:,i] = beta[:,id-1]
    eps = np.random.gumbel(scale=1,size=(11192,))
    P = (X[:,2:]@ beta_choice)[:,0] + eps
#     try:
#         p = P[0:4]
#         check = p/np.sum(p)
#         assert(np.sum(check) == 1. or np.sum(check) == 1)
#     except AssertionError:
#         raise AssertionError('Kansen van eerste aankoop sommeren niet naar 1 %f'%(np.sum(check)))

    Y = []
    for i in range(0,len(P), 4):
        choice = np.argmax(P[i:i+4])
#         p = P[i:i+4]
#         check = p/np.sum(p)
#         print(np.sum(check))
        Y.append(int(choice))
    return np.array(Y)
probs(X,'QMC(75)',)

In [14]:
qmc75 = summarize_data('QMC(75)')
print(qmc75[0].x.shape)

(6,)


In [ ]:
def get_results(model_data):
    #model data should be in list format containing OptimizeResult types
    results ={}
    theta_est = np.zeros((6,))
    mape = 0
    for run in model_data:
        theta_est += run.x/len(mode_data)
    
    